In [1]:
from openff.qcsubmit.results import TorsionDriveResultCollection
from openff.toolkit.topology import Molecule
import logging
from rdkit import RDLogger

In [2]:
logging.getLogger("rdkit").setLevel(logging.ERROR)
RDLogger.DisableLog('rdApp.*')

In [3]:
# Using qcsubmit to create a torsiondrive dataset for the central bridging C-C bonds
from openff.qcsubmit.common_structures import QCSpec, SCFProperties
from openff.qcsubmit.factories import TorsiondriveDatasetFactory
from openff.qcsubmit.workflow_components import (
    Scan1D, ScanEnumerator, StandardConformerGenerator
)
import numpy as np
from qcelemental.models.results import WavefunctionProtocolEnum
from qcportal.models.common_models import DriverEnum

# Specifying the theory level for the calculations
# Anything computable by QCEngine can be specified and they all can be included at the same time
# Here I am adding QM and SEQM methods
qc_specifications = {
                    "qm_spec": 
                            QCSpec(method='b3lyp-d3bj', 
                               basis='dzvp', 
                               program='psi4', 
                               spec_name='qm_spec', 
                               spec_description='Project quantum chemistry specification',
                               implicit_solvent=None, 
                               maxiter=200, 
                               scf_properties=[SCFProperties.Dipole, SCFProperties.Quadrupole, SCFProperties.WibergLowdinIndices, 
                                               SCFProperties.MayerIndices, SCFProperties.MBISCharges],
                               keywords=None,
                              ),
                    "gfn2xtb": 
                        QCSpec(method="gfn2xtb",
                            basis=None,
                            program="xtb",
                            spec_name="gfn2xtb",
                            spec_description="A default spec for gfn2xtb",
                            implicit_solvent=None,
                            maxiter=200,
                            scf_properties=[SCFProperties.Dipole, SCFProperties.Quadrupole, SCFProperties.WibergLowdinIndices, 
                                            SCFProperties.MayerIndices, SCFProperties.MBISCharges],
                            keywords=None,
                            ),
                    }
        
# Torsion driven is tagged with a smarts pattern here. 
# It can also be chosen by passing the dihedral indices using `openff.qcsubmit.common_structures.TorsionIndexer()`. 
# A sample submission can be found here, https://github.com/openforcefield/qca-dataset-submission/blob/master/submissions/2021-04-09-OpenFF-Gen3-Torsion-Set-v1.0/generate-dataset.ipynb

torsion_drive_factory = TorsiondriveDatasetFactory(
                            qc_specifications=qc_specifications,
                            workflow=[
                                ScanEnumerator(
                                    torsion_scans=[
                                        Scan1D(
                                            smarts1="[#6X3H1:1]~[#6X3:2](~[#6X3H1])-[#6X3:3](~[#6X3H1])~[#6X3H1:4]",
                                            scan_range1=(-180, 180),
                                            scan_increment=[15]
                                        )
                                    ]
                                ),
                                StandardConformerGenerator(toolkit="openeye", max_conformers=10)
                            ],
)


# Biaryl molecules created from constructure package with different substituents and different sizes of rings are passed through the smi file.
# Molecules here can be created from SDF file and other supported formats as well.
dataset = torsion_drive_factory.create_dataset(
    molecules='biaryl_set.smi',
    dataset_name="Biaryl Torsion Drives",
    description="A dataset containing biaryls with no ortho substituents around the bridging C-C bond",
    tagline="A custom biaryl set without ortho substituents",
)

dataset.metadata.submitter = 'pavankum'
dataset.metadata.long_description_url = (
    'https://custom_submission.com'
)

Preparation                   : 100%|███████████| 60/60 [00:01<00:00, 49.82it/s]


In [4]:
# More compute specifications can be added post dataset creation as well.

# add ANI2x (check supported elements and charged molecules on the dataset, otherwise some calculatons may fail)
dataset.add_qc_spec(method="ani2x", basis=None, program="torchani", spec_name="ani2x", spec_description="A default spec for ani2x")

# MM force fields can be added as well that are calculated with OpenMM
dataset.add_qc_spec(method="openff-2.0.0", basis="smirnoff", spec_name="openff-2.0.0", spec_description="A default spec for openff-2.0.0", program="openmm")
dataset.add_qc_spec(method="gaff-2.11", basis="antechamber", spec_name="gaff-2.11", spec_description="A default spec for gaff-2.11", program="openmm")

In [5]:
from rdkit.Chem import Descriptors

In [6]:
confs = np.array([mol.n_conformers for mol in dataset.molecules])
molecular_weights = np.array([Descriptors.ExactMolWt(mol.to_rdkit()) for mol in dataset.molecules])
unique_formal_charges = np.unique([mol.total_charge / mol.total_charge.unit for mol in dataset.molecules])

print('Number of unique molecules        {:d}'.format(dataset.n_molecules))
print('Number of filtered molecules      {:d}'.format(dataset.n_filtered))
print('Number of torsion drives          {:d}'.format(dataset.n_records))
print('Number of conformers min mean max {:3d} {:6.2f} {:3d}'.format(confs.min(), confs.mean(), confs.max()))
print('Molecular weight min mean max     {:6.2f} {:6.2f} {:6.2f}'.format(
    molecular_weights.min(), molecular_weights.mean(), molecular_weights.max()))
print('Charges                          ', sorted(unique_formal_charges), '\n')

print(dataset.metadata.dict(), '\n')

for spec, obj in dataset.qc_specifications.items():
    print("Spec:", spec)
    print(obj.dict(),'\n')

Number of unique molecules        60
Number of filtered molecules      0
Number of torsion drives          60
Number of conformers min mean max   1   1.00   1
Molecular weight min mean max     143.07 209.20 266.01
Charges                           [-1.0, 0.0, 1.0] 

{'submitter': 'pavankum', 'creation_date': datetime.date(2022, 6, 8), 'collection_type': 'TorsionDriveDataset', 'dataset_name': 'Biaryl Torsion Drives', 'short_description': 'A custom biaryl set without ortho substituents', 'long_description_url': HttpUrl('https://custom_submission.com', scheme='https', host='custom_submission.com', tld='com', host_type='domain', port='443'), 'long_description': 'A dataset containing biaryls with no ortho substituents around the bridging C-C bond', 'elements': {'O', 'S', 'C', 'H', 'N'}} 

Spec: qm_spec
{'method': 'b3lyp-d3bj', 'basis': 'dzvp', 'program': 'psi4', 'spec_name': 'qm_spec', 'spec_description': 'Project quantum chemistry specification', 'store_wavefunction': 'none', 'implicit_sol

In [7]:
# export the final dataset

dataset.export_dataset("dataset.json.bz2")
dataset.molecules_to_file("dataset.smi", "smi")

dataset.visualize("dataset.pdf", columns=4)

# Sample dataset preparation notebooks

Many more sample dataset preparation notebooks using openff-qcsubmit can be found under https://github.com/openforcefield/qca-dataset-submission/blob/master/submissions. \
Some of the recent submissions for other types of datasets (other than torsiondrive shown above)
- OptimizationDataset: https://github.com/openforcefield/qca-dataset-submission/blob/master/submissions/2021-12-21-OpenFF-Gen2-Optimization-Set-Protomers/Dataset_Generation.ipynb
- BasicDataset: https://github.com/openforcefield/qca-dataset-submission/blob/master/submissions/2021-11-15-QMDataset-DES-monomers-single-points/Dataset_Generation.ipynb